In [56]:
"""
Classes and methods for working with nondeterministic finite automata.
Source: https://github.com/caleb531/automata
"""

import copy
import abc
import math
from collections import defaultdict

class AutomatonException(Exception):
    """The base class for all automaton-related errors."""
    pass


class InvalidStateError(AutomatonException):
    """A state is not a valid state for this automaton."""
    pass


class InvalidSymbolError(AutomatonException):
    """A symbol is not a valid symbol for this automaton."""
    pass


class MissingStateError(AutomatonException):
    """A state is missing from the automaton definition."""
    pass


class MissingSymbolError(AutomatonException):
    """A symbol is missing from the automaton definition."""
    pass


class InitialStateError(AutomatonException):
    """The initial state fails to meet some required condition."""
    pass


class FinalStateError(AutomatonException):
    """A final state fails to meet some required condition."""
    pass


class RejectionException(AutomatonException):
    """The input was rejected by the automaton."""
    pass


class Automaton(metaclass=abc.ABCMeta):
    """An abstract base class for all automata, including Turing machines."""

    @abc.abstractmethod
    def __init__(self):
        """Initialize a complete automaton."""
        raise NotImplementedError

    @abc.abstractmethod
    def validate(self):
        """Return True if this automaton is internally consistent."""
        raise NotImplementedError

    @abc.abstractmethod
    def read_input_stepwise(self, input_str):
        """Return a generator that yields each step while reading input."""
        raise NotImplementedError

    def read_input(self, input_str):
        """
        Check if the given string is accepted by this automaton.

        Return the automaton's final configuration if this string is valid.
        """
        validation_generator = self.read_input_stepwise(input_str)
        for config in validation_generator:
            pass
        return config

    def accepts_input(self, input_str):
        """Return True if this automaton accepts the given input."""
        try:
            self.read_input(input_str)
            return True
        except RejectionException:
            return False

    def _validate_initial_state(self):
        """Raise an error if the initial state is invalid."""
        if self.initial_state not in self.states:
            raise InvalidStateError(
                '{} is not a valid initial state'.format(self.initial_state))

    def _validate_initial_state_transitions(self):
        """Raise an error if the initial state has no transitions defined."""
        if self.initial_state not in self.transitions:
            raise MissingStateError(
                'initial state {} has no transitions defined'.format(
                    self.initial_state))

    def _validate_final_states(self):
        """Raise an error if any final states are invalid."""
        invalid_states = self.final_states - self.states
        if invalid_states:
            raise InvalidStateError(
                'final states are not valid ({})'.format(
                    ', '.join(str(state) for state in invalid_states)))

    def copy(self):
        """Create a deep copy of the automaton."""
        return self.__class__(**vars(self))

    def __eq__(self, other):
        """Check if two automata are equal."""
        return vars(self) == vars(other)


class FA(Automaton, metaclass=abc.ABCMeta):
    """An abstract base class for finite automata."""
    pass

def ddict2dict(d):
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = ddict2dict(v)
    return dict(d)

class NFA(FA):
    """A nondeterministic finite automaton."""
    def __init__(self, *, states, input_symbols, transitions,
                 initial_state, final_states):
        """Initialize a complete NFA."""
        self.states = states.copy()
        self.input_symbols = input_symbols.copy()
        self.transitions = copy.deepcopy(transitions)
        self.initial_state = initial_state
        self.final_states = final_states.copy()
        self.validate()

    def _validate_transition_invalid_symbols(self, start_state, paths):
        """Raise an error if transition symbols are invalid."""
        for input_symbol in paths.keys():
            if input_symbol not in self.input_symbols and input_symbol != '':
                raise InvalidSymbolError(
                    'state {} has invalid transition symbol {}'.format(
                        start_state, input_symbol))

    def _validate_transition_end_states(self, start_state, paths):
        """Raise an error if transition end states are invalid."""
        for end_states in paths.values():
            for end_state in end_states:
                if end_state not in self.states:
                    raise InvalidStateError(
                        'end state {} for transition on {} is '
                        'not valid'.format(end_state, start_state))

    def validate(self):
        """Return True if this NFA is internally consistent."""
        for start_state, paths in self.transitions.items():
            self._validate_transition_invalid_symbols(start_state, paths)
            self._validate_transition_end_states(start_state, paths)
        self._validate_initial_state()
        self._validate_initial_state_transitions()
        self._validate_final_states()
        return True

    def _get_lambda_closure(self, start_state):
        """
        Return the lambda closure for the given state.

        The lambda closure of a state q is the set containing q, along with
        every state that can be reached from q by following only lambda
        transitions.
        """
        stack = []
        encountered_states = set()
        stack.append(start_state)

        while stack:
            state = stack.pop()
            if state not in encountered_states:
                encountered_states.add(state)
                if '' in self.transitions[state]:
                    stack.extend(self.transitions[state][''])

        return encountered_states

    def _get_next_current_states(self, current_states, input_symbol):
        """Return the next set of current states given the current set."""
        next_current_states = set()

        for current_state in current_states:
            symbol_end_states = self.transitions[current_state].get(
                input_symbol)
            if symbol_end_states:
                for end_state in symbol_end_states:
                    next_current_states.update(
                        self._get_lambda_closure(end_state))

        return next_current_states

    def _check_for_input_rejection(self, current_states):
        """Raise an error if the given config indicates rejected input."""
        if not (current_states & self.final_states):
            raise RejectionException(
                'the NFA stopped on all non-final states ({})'.format(
                    ', '.join(str(state) for state in current_states)))

    def read_input_stepwise(self, input_str):
        """
        Check if the given string is accepted by this NFA.

        Yield the current configuration of the NFA at each step.
        """
        current_states = self._get_lambda_closure(self.initial_state)

        yield current_states
        for input_symbol in input_str:
            current_states = self._get_next_current_states(
                current_states, input_symbol)
            yield current_states

        self._check_for_input_rejection(current_states)

    def unroll(self, n: int):
        """
        Builds A_unroll with n levels
        to estimate |L(F^n)|
        """
        # for each state q ∈ Q, include copies q_0 , q_1 , ..., q n in A unroll
        new_states = {f"{q}_{i}" for q in self.states for i in range(n+1)}
        # for each transition (p, a, q) ∈ ∆ and i ∈ {0, 1, . . . , n − 1}, include
        # transition (p_i, a, q_i+1) in A unroll
        new_transitions = defaultdict(lambda: defaultdict(set))
        for p, trans in self.transitions.items():
            for a, qs in trans.items():
                for q in qs:
                    for i in range(n):
                        new_transitions[f"{p}_{i}"][a].add(f"{q}_{i+1}")
        new_final_states = {f"{q}_{n}" for q in self.final_states}
        print(new_states)
        print(ddict2dict(new_transitions))
        print(new_final_states)
        
        

In [58]:
nfa = NFA(
    states={'q0', 'q1', 'q2'},
    input_symbols={'0', '1'},
    transitions={
        'q0': {'0': {'q1'}},
        # Use '' as the key name for empty string (lambda/epsilon) transitions
        'q1': {'0': {'q1'}, '1': {'q0', 'q2'}},
        'q2': {'1': {'q0'}}
    },
    initial_state='q0',
    final_states={'q1'}
)

In [31]:
def bruteforce_save_all(nfa: NFA, n: int):
    accepted_strings = []
    for i in range(2**n):
        string_i = bin(i)[2:].zfill(n)
        if nfa.accepts_input(string_i):
            accepted_strings.append(string_i)
    return accepted_strings

def bruteforce_count_only(nfa: NFA, n: int):
    accepted_count = 0
    for i in range(2**n):
        string_i = bin(i)[2:].zfill(n)
        if nfa.accepts_input(string_i):
            accepted_count += 1
    return accepted_count

In [35]:
%%timeit
bruteforce_count_only(nfa, 16)

387 ms ± 4.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
bruteforce_count_only(nfa, 16)

5768

In [59]:
nfa.unroll(4)

{'q2_1', 'q2_2', 'q2_0', 'q2_3', 'q1_2', 'q2_4', 'q0_2', 'q1_0', 'q0_3', 'q0_4', 'q1_1', 'q0_1', 'q1_3', 'q0_0', 'q1_4'}
{'q0_0': {'0': {'q1_1'}}, 'q0_1': {'0': {'q1_2'}}, 'q0_2': {'0': {'q1_3'}}, 'q0_3': {'0': {'q1_4'}}, 'q1_0': {'0': {'q1_1'}, '1': {'q2_1', 'q0_1'}}, 'q1_1': {'0': {'q1_2'}, '1': {'q0_2', 'q2_2'}}, 'q1_2': {'0': {'q1_3'}, '1': {'q0_3', 'q2_3'}}, 'q1_3': {'0': {'q1_4'}, '1': {'q0_4', 'q2_4'}}, 'q2_0': {'1': {'q0_1'}}, 'q2_1': {'1': {'q0_2'}}, 'q2_2': {'1': {'q0_3'}}, 'q2_3': {'1': {'q0_4'}}}
{'q1_4'}
